In [1]:
from library import *

class Equation(object): # can represent equation (expression = 0) OR expression
    def __init__(self, term_list, coeffs): # terms are LibraryTerms, coeffs are real numbers
        content = zip(term_list, coeffs)
        sorted_content = sorted(content, key=lambda x:x[0])
        self.term_list = [e[0] for e in sorted_content]
        self.coeffs = [e[1] for e in sorted_content]
        self.rank = term_list[0].rank
        # might want to define complexity function
      
    def __add__(self, other):
        if isinstance(other, Equation):
            return Equation(self.term_list + other.term_list, self.coeffs + other.coeffs)
        else:
            raise ValueError(f"Second argument {other}) is not an equation.")
    
    def __repr__(self):
        repstr = [str(coeff) + ' * ' + str(term)+' + ' for term, coeff in zip(self.term_list, self.coeffs)]
        return reduce(add, repstr)[:-3]
    
    def __str__(self):
        return self.__repr__()+" = 0"
    
    def __eq__(self, other):
        for term, ot in zip(self.term_list, other.term_list):
            if term != ot:
                return False
        for coeff, ot in zip(self.coeffs, other.coeffs):
            if term != ot:
                return False
        return True
    
    def eliminate_complex_term(self):
        lhs = max(self.term_list, key=lambda t:t.complexity)
        lhs_ind = self.term_list.index(lhs)
        new_term_list = self.term_list[:lhs_ind]+self.term_list[lhs_ind+1:]
        new_coeffs = self.coeffs[:lhs_ind]+self.coeffs[lhs_ind+1:]
        new_coeffs = [c/self.coeffs[lhs_ind] for c in new_coeffs]
        rhs = Equation(new_term_list, new_coeffs)
        return lhs, rhs

# need to implement:
# 2. term builder-upper - enough to just build up from the most complex term which we exclude,
#         not necessary to build up whole equation in this way [not sure what I meant when I wrote this!]
# 3. iterative term selecter + model builder (to find models up to complexity K)
# 4. term excluder (based on identified models)